In [1]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time


In [2]:
SEASONS = list(range(2018,2024))


In [3]:
DATA_DIR = 'data'
MONTHS_DIR = os.path.join(DATA_DIR, 'months')
SCORES_DIR = os.path.join(DATA_DIR, 'scores')

In [4]:
async def get_html(url, selector, sleep=5, retries = 3):
    """
    Fetches the HTML content of a specified element from a webpage using Playwright.
    Args:
        url (str): The URL of the webpage to fetch the HTML from.
        selector (str): The CSS selector of the element to fetch the HTML content from.
        sleep (int, optional): The base number of seconds to wait between retries. Defaults to 5.
        retries (int, optional): The number of times to retry fetching the HTML in case of failure. Defaults to 3.
    Returns:
        str: The HTML content of the specified element, or None if the fetch fails after the specified retries.
    Raises:
        PlaywrightTimeout: If the request times out.
    """

    html = None
    for i in range(1,retries+1):
        time.sleep(sleep * i) #makes sure we dont scrape too fast and we dont get banned.

        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout on {url}")
            continue
        else: 
            break
    return html


In [5]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, '#content .filter')
    soup = BeautifulSoup(html)
    links = soup.find_all('a')
    href = [l['href'] for l in links]
    months_pages = [f'https://www.basketball-reference.com{l}' for l in href]

    for url in months_pages:
        save_path = os.path.join(MONTHS_DIR, url.split('/')[-1]) #naming by month
        if os.path.exists(save_path):
            print(f"Skipping {url}")
            continue

        html = await get_html(url, '#all_schedule')
        with open(save_path, 'w+') as f:
            f.write(html)

In [9]:
for season in SEASONS:
    await scrape_season(season)

2017-18 NBA Schedule | Basketball-Reference.com
Skipping https://www.basketball-reference.com/leagues/NBA_2018_games-october.html
Skipping https://www.basketball-reference.com/leagues/NBA_2018_games-november.html
Skipping https://www.basketball-reference.com/leagues/NBA_2018_games-december.html
Skipping https://www.basketball-reference.com/leagues/NBA_2018_games-january.html
Skipping https://www.basketball-reference.com/leagues/NBA_2018_games-february.html
Skipping https://www.basketball-reference.com/leagues/NBA_2018_games-march.html
Skipping https://www.basketball-reference.com/leagues/NBA_2018_games-april.html
Skipping https://www.basketball-reference.com/leagues/NBA_2018_games-may.html
Timeout on https://www.basketball-reference.com/leagues/NBA_2018_games-june.html
2017-18 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
Skipping https://www.basketball-reference.com/leagues/NBA_2019_games-october.html
Skipping https://www.basketball-reference.

TypeError: write() argument must be str, not None